In [535]:
'''
Create variable string. 
'''

def create_variable_string(variable_codes): 
    
    # Initialize empty variable string 
    variable_string = r"NAME,B01001_001E"

    # Iterate through filtered dictionary 
    for code in variable_codes: 
        variable_string = variable_string + ',' + code
    
    return variable_string 

'''
Use the geographic field attributes of the feature classes to create a list of urls 
'''
def construct_api_urls(geo_features, api_key, variable_string): 
    
    api_urls = []
    
    with arcpy.da.SearchCursor(geo_features, ['FIPS']) as cursor: 
        for row in cursor: 
            state       = row[0][0:2]
            county      = row[0][2:5]
            tract       = row[0][5:11]
            block_group = row[0][11]

            var_clause    = fr"https://api.census.gov/data/2022/acs/acs5?get={variable_string}"
            geo_clause    = fr"&for=block%20group:{block_group}&in=state:{state}%20county:{county}%20tract:{tract}"
            key_clause    = fr"&key={api_key}"

            api_url       = var_clause + geo_clause + key_clause
            api_urls.append(api_url)
        
    return api_urls

'''
Iterate through API URLs and receive dictionary of responses 
'''

def call_census_api(api_urls):
    
    json_list = [] 
    url_count = 0
    
    # Initialize time 
    time_1  = time.time() 
    
    # Send API call 
    print(len(api_urls))
    print('Continue?')
    
    x = input()
    if x == 'no': 
        quit()  
    else: 
        pass 
    
    for url in api_urls: 
        url_count += 1 
        
        print(f'Call #{url_count} / {len(api_urls)}')
        response = requests.get(url)
        print(">>> Sent request")

        print("Checking status:")
        if response.status_code == 200: 
            json_list.append(response.json())
            print(">>> Good")

        else: 
            print('>>> Error: ', response.status_code)
            
        # Measure and print elapsed time 
        time_2  = time.time() 
        elapsed = round(time_2 - time_1, 2) 
        print(f'Elapsed time:')
        print(f">>> {elapsed}")
        print(" ")
    
    return json_list

In [536]:
'''
Master Function 
'''

def retrieve_acs_json(variable_codes, geo_features, api_key): 
    
    variable_string = create_variable_string(variable_codes)
    api_urls        = construct_api_urls(geo_features, api_key, variable_string)
    json_list       = call_census_api(api_urls)
    
    return json_list

'''
Parameters
'''

variable_codes   
geo_features     = r'C:\DALE\Personal\Minnesota\GIS Projects\ACS\Output Data\Block Group Shapefiles\Chicago\Chicago_5.shp'
api_key          = r'1749dc1d87964116107a80cd7d76fca300dda59f'

'''
Run
'''
json_list = retrieve_acs_json(variable_codes, geo_features, api_key)

468
Continue?
d
Call #1 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 7.39
 
Call #2 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 8.1
 
Call #3 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 8.83
 
Call #4 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 9.61
 
Call #5 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 10.35
 
Call #6 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 11.13
 
Call #7 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 11.92
 
Call #8 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 12.72
 
Call #9 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 14.54
 
Call #10 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 15.33
 
Call #11 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 16.05

Call #91 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 76.53
 
Call #92 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 77.25
 
Call #93 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 77.99
 
Call #94 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 78.71
 
Call #95 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 79.43
 
Call #96 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 80.17
 
Call #97 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 80.99
 
Call #98 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 81.73
 
Call #99 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 82.44
 
Call #100 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 83.16
 
Call #101 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 83.88

>>> Error:  400
Elapsed time:
>>> 140.37
 
Call #180 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 141.12
 
Call #181 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 141.84
 
Call #182 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 142.56
 
Call #183 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 143.28
 
Call #184 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 143.99
 
Call #185 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 144.69
 
Call #186 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 145.4
 
Call #187 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 146.14
 
Call #188 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 146.9
 
Call #189 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 147.61
 
Call #190 / 468
>>> Sent requ

>>> Error:  400
Elapsed time:
>>> 205.5
 
Call #268 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 206.33
 
Call #269 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 207.13
 
Call #270 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 207.88
 
Call #271 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 208.62
 
Call #272 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 209.37
 
Call #273 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 210.09
 
Call #274 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 210.8
 
Call #275 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 211.5
 
Call #276 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 212.2
 
Call #277 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 212.91
 
Call #278 / 468
>>> Sent reques

Elapsed time:
>>> 273.46
 
Call #356 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 274.23
 
Call #357 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 274.96
 
Call #358 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 275.69
 
Call #359 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 276.48
 
Call #360 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 277.29
 
Call #361 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 278.06
 
Call #362 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 278.94
 
Call #363 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 279.72
 
Call #364 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 280.46
 
Call #365 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 281.18
 
Call #366 / 468
>>> Sent request
Checking s

>>> 338.32
 
Call #444 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 339.06
 
Call #445 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 339.79
 
Call #446 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 340.58
 
Call #447 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 341.32
 
Call #448 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 342.02
 
Call #449 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 342.77
 
Call #450 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 343.49
 
Call #451 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 344.21
 
Call #452 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 344.95
 
Call #453 / 468
>>> Sent request
Checking status:
>>> Error:  400
Elapsed time:
>>> 345.66
 
Call #454 / 468
>>> Sent request
Checking status:
>>> Err